# Connected Components

In this notebook, we will use cuGraph to compute weakly and strongly connected components of a graph and display some useful information about the resulting components.

Notebook Credits
* Original Authors: Kumar Aatish
* Last Edit: 08/05/2019

RAPIDS Versions: 0.9.0    

Test Hardware

* GV100 32G, CUDA 10,1



## Introduction

To compute WCC for a graph in cuGraph we use:
**cugraph.weakly_connected_components(G)**

To compute SCC for a graph in cuGraph we use:
**cugraph.strongly_connected_components(G)**

Both of these calls have identical api:  

Input
* __G__: cugraph.Graph object

Returns 
* __df__:  a cudf.DataFrame object with two columns:
    * df['labels'][i]: Gives the label id of the i'th vertex
    * df['vertices'][i]: Gives the vertex id of the i'th vertex



## cuGraph 0.9 Notice 
cuGraph version 0.9 has some limitations:
* Only Int32 Vertex ID are supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations.

### Test Data
We will be using the Netscience dataset :  
*M. E. J. Newman, Finding community structure in networks using the eigenvectors of matrices, Preprint physics/0605087 (2006)*

Netscience Adjacency Matrix               |NetScience Strongly Connected Components
:-----------------------------------------|--------------------------------------------------:
![](./img/netscience.png "Credit : https://www.cise.ufl.edu/research/sparse/matrices/Newman/netscience") | ![](./img/netscience_scc.png "Credit : https://www.cise.ufl.edu/research/sparse/matrices/Newman/netscience")

Matrix pictures by [cspy](http://www.cise.ufl.edu/research/sparse/CSparse/CSparse/MATLAB/CSparse/cspy.m), a MATLAB function in the [CSparse](http://www.cise.ufl.edu/research/sparse/CSparse) package.  
Matrix graphs by Yifan Hu, AT&T Labs Visualization Group.

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from collections import OrderedDict

### 1. Read graph data from file

In [2]:
datafile='./data/netscience.csv'
cols = ["src", "dst", "wgt"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32"),
        ("wgt", "float32")
        ])
usecols = ["src", "dst"]
gdf = cudf.read_csv(datafile, names=cols, usecols = usecols, delimiter=' ', dtype=list(dtypes.values()) )
gdf.head().to_pandas()

,src,dst
0,1,0
1,1084,0
2,946,1
3,1084,1
4,3,2


### 2. Create a Graph from an edge list

In [3]:
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

### 3a. Call Weakly Connected Components

In [4]:
df = cugraph.weakly_connected_components(G)
print(df)

    labels  vertices
0       1         0
1       1         1
2       3         2
3       3         3
4       3         4
5       3         5
6       3         6
7       8         7
8       8         8
9       8         9
[1579 more rows]


#### Get total number of weakly connected components

In [5]:
label_gby = df.groupby('labels')
label_count = label_gby.count()
print("Total number of components found : ", len(label_count))

Total number of components found :  396


#### Get size of the largest weakly connected component

In [6]:
largest_component = label_count.nlargest(n = 1, columns = 'vertices')
print("Size of the largest component is found to be : ", largest_component['vertices'][0])

Size of the largest component is found to be :  379


#### Output vertex ids belonging to a weakly connected component label

In [7]:
expr = "labels == 1"
component = df.query(expr)
print("Vertex Ids that belong to component label 1 : ")
print(component)

Vertex Ids that belong to component label 1 : 
   labels  vertices
0       1         0
1       1         1
946       1       946
1084       1      1084


### 3b. Call Strongly Connected Components

In [8]:
df = cugraph.strongly_connected_components(G)
print(df)

    labels  vertices
0       0         0
1       0         1
2       2         2
3       2         3
4       2         4
5       2         5
6       2         6
7       7         7
8       7         8
9       7         9
[1579 more rows]


#### Get total number of strongly connected components

In [9]:
label_gby = df.groupby('labels')
label_count = label_gby.count()
print("Total number of components found : ", len(label_count))

Total number of components found :  396


#### Get size of the largest strongly connected component

In [10]:
largest_component = label_count.nlargest(n = 1, columns = 'vertices')
print("Size of the largest component is found to be : ", largest_component['vertices'][0])

Size of the largest component is found to be :  379


#### Output vertex ids belonging to a strongly connected component label

In [11]:
expr = "labels == 2"
component = df.query(expr)
print("Vertex Ids that belong to component label 2 : ")
print(component)

Vertex Ids that belong to component label 2 : 
   labels  vertices
2       2         2
3       2         3
4       2         4
5       2         5
6       2         6


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___